In [2]:
!pip install everypolitician

In [3]:
from everypolitician import EveryPolitician
import json
from bs4 import BeautifulSoup
import requests
ep = EveryPolitician()


try:
    with open("countries.json", "r") as file:
        countryJSON = json.load(file)
        print(countryJSON)
except json.JSONDecodeError as e:
    print(f"Failed to decode JSON: {e}")
except Exception as e:
    print(f"An error occurred: {e}")




[{'key': '', 'doc_count': 632}, {'key': 'Australia', 'doc_count': 145}, {'key': 'Indonesia', 'doc_count': 95}, {'key': 'China', 'doc_count': 94}, {'key': 'Malaysia', 'doc_count': 74}, {'key': 'India', 'doc_count': 71}, {'key': 'Vietnam', 'doc_count': 42}, {'key': 'Singapore', 'doc_count': 31}, {'key': 'Japan', 'doc_count': 28}, {'key': 'Qatar', 'doc_count': 24}]


In [4]:
countryList = [country['key'] for country in countryJSON]
countryList = countryList[1:] #remove the first element because its empty
print(countryList)




['Australia', 'Indonesia', 'China', 'Malaysia', 'India', 'Vietnam', 'Singapore', 'Japan', 'Qatar']


In [5]:
# get the list of politicians for each country
politicians_by_country = {}

for country in countryList:
    try:
        country_legislature = ep.country(country).lower_house()
        politicians = country_legislature.popolo().persons
        politicians_by_country[country] = [politician.name for politician in politicians]
    except Exception as e:
        print(f"Failed to retrieve data for {country}: {e}")

print(politicians_by_country)





Failed to retrieve data for Qatar: Couldn't find the country with slug 'Qatar'
{'Australia': ['De-Anne Kelly', 'Percival Millar', 'John Alexander', 'Michael Ronaldson', 'Peter Andren', 'David Simmons', 'Warwick Smith', 'Alasdair Webster', 'Jim Turnour', 'Kym Richardson', 'Margaret May', 'Alby Schultz', 'Tanya Plibersek', 'Peter Costello', 'Robert Chynoweth', 'Frederick Chaney', 'Bob Horne', 'Ian Robinson', 'Wallace Fife', 'Andrew Wallace', 'Richard Evans', 'Bruce Goodluck', 'Nicolle Flint', 'Ken Wyatt', 'Cameron Thompson', 'Ross Cameron', 'John Dawkins', 'Janice Crosio', 'Silvia Smith', 'Jason Clare', 'Dick Adams', 'Peter Lindsay', 'Ben Morton', 'Fiona Scott', 'William Taylor', 'Jackie Kelly', 'Janelle Saffin', "Gavan O'Connor", 'Sharryn Jackson', 'Ann Corcoran', 'Ian McLachlan', 'Maria Vamvakinou', 'Martyn Evans', 'Gary Punch', 'Michael Mackellar', 'Peter Nugent', 'Allan Morris', 'David Beddall', 'Richard Klugman', 'Ian Cameron', 'Nicholas Reid', 'Duncan Kerr', 'Kevin Andrews', 'Antho

we need a dedicated function to scrape china, since the wikipedia site is zh.wikipedia.org instead of the rest, which is en.wikipedia.org

In [17]:
# test function to scrape Australia
def test_scrape_australian_politicians_details():
    australian_politicians = politicians_by_country.get('Australia', [])[:10]  # Get the first 10 Australian politicians
    politician_details = []

    for politician in australian_politicians:
        try:
            # Construct the URL for each politician's Wikipedia page
            url = f'https://en.wikipedia.org/wiki/{politician.replace(" ", "_")}'  # Replace spaces with underscores for URL
            response = requests.get(url)
            response.raise_for_status()  # Raises an HTTPError for bad responses

            # Parse the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract specific details based on the provided CSS selectors
            details = {}

            # Extract the link to the politician's Wikipedia page
            details['Wikipedia Link'] = url

            # Extract the image link
            image_link = soup.select_one('#mw-content-text > div.mw-content-ltr.mw-parser-output > table > tbody > tr:nth-child(2) > td > span > a')
            if image_link and image_link.has_attr('href'):
                details['Image Link'] = image_link['href']

            for sup in soup.find_all('sup'):
                sup.decompose()
            description = []
            for p in soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox.vcard ~ p'):
                description.append(p.text.strip())
            print(description)
            print(f"Length of description for {politician}: {len(' '.join(description))} characters")
            details['Description'] = ' '.join(description)
            politician_details.append({politician: details})
        except requests.RequestException as e:
            print(f"HTTP Request failed for {politician}: {e}")
        except Exception as e:
            print(f"An error occurred while scraping {politician}: {e}")

    return politician_details

# Call the test function to scrape details of Australian politicians
australian_politicians_details = test_scrape_australian_politicians_details()
import pandas as pd
from IPython.display import display

# Convert the list of Australian politician details into a DataFrame
df_australian_politicians = pd.DataFrame([details for sublist in australian_politicians_details for details in sublist.values()])
display(df_australian_politicians)

https://excalidraw.com/#room=7eb6c49243eb884d3cb6,eAG_T9ANXEWLJ8DcXk1aOw

['De-Anne Margaret Kelly (née Park; born 21 March 1954) is an Australian former politician.', 'Kelly was a National Party member of the Australian House of Representatives from March 1996 until November 2007, representing the Division of Dawson, Queensland.', 'She was the first female member of the National Party to win a seat in the House of Representatives.', "Kelly was born in Rockhampton, Queensland to parents Ian Park and Margaret Park (née Bauman). Kelly's uncle was Kerrod Park, who was known for his involvement with local government in Queensland, having served as the chairman of Duaringa Shire Council from 1973 to 1994.", 'Kelly grew up on a cattle property where she obtained her primary education from her mother who taught her at the kitchen table. She then attended high school in Rockhampton and after winning a scholarship to study electrical engineering, Kelly attended the University of Queensland.', 'After graduating, Kelly worked as an engineer before she bought a small ma

,Wikipedia Link,Image Link,Description
0,https://en.wikipedia.org/wiki/De-Anne_Kelly,/wiki/File:De-Anne_Kelly_2006.jpg,De-Anne Margaret Kelly (née Park; born 21 Marc...
1,https://en.wikipedia.org/wiki/Percival_Millar,NaN,Percival Clarence Millar AM (né Turbill; 15 Ju...
2,https://en.wikipedia.org/wiki/John_Alexander,NaN,
3,https://en.wikipedia.org/wiki/Michael_Ronaldson,/wiki/File:Michael_Ronaldson.jpg,Michael John Clyde Ronaldson (born 13 February...
4,https://en.wikipedia.org/wiki/Peter_Andren,NaN,Peter James Andren AM (28 August 1946 – 3 Nove...
5,https://en.wikipedia.org/wiki/David_Simmons,NaN,
6,https://en.wikipedia.org/wiki/Warwick_Smith,NaN,
7,https://en.wikipedia.org/wiki/Alasdair_Webster,NaN,Alasdair Paine Webster OAM (born 12 February 1...
8,https://en.wikipedia.org/wiki/Jim_Turnour,NaN,James Pearce Turnour (born 7 April 1966) is a ...
9,https://en.wikipedia.org/wiki/Kym_Richardson,NaN,Kym Charles Richardson (born 16 March 1958) is...


In [7]:



def scrape_china():
    try:
        # Use the specific URL for Chinese Wikipedia
        url = 'https://zh.wikipedia.org/wiki/政治人物列表'
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        politicians = []

        # Assuming the politicians are listed in a table with class 'wikitable'
        for row in soup.find_all('table', class_='wikitable'):
            for politician in row.find_all('tr')[1:]:  # Skip header row
                name = politician.find_all('td')[0].text.strip()
                politicians.append(name)

        print(f"Scraped {len(politicians)} politicians from China.")
        return politicians
    except requests.RequestException as e:
        print(f"HTTP Request failed: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to scrape Chinese politicians
chinese_politicians = scrape_china()
print(chinese_politicians)


HTTP Request failed: 404 Client Error: Not Found for url: https://zh.wikipedia.org/wiki/%E6%94%BF%E6%B2%BB%E4%BA%BA%E7%89%A9%E5%88%97%E8%A1%A8
None
